# Laberinto / Búsqueda con información

- Poder modificar el tamaño del tablero
- Colocar obstáculos aleatorios
- Poder tener más de una meta (que el agente sabrá cuantas metas habrá en el tablero)
- Inicio del agente de manera aleatorio

In [19]:
from math import floor, sqrt
from colorama import Fore

In [20]:
def color_text(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)
    
# map icons
CHAR_WALL = '⬛️'
CHAR_EMPTY = '🐾'
CHAR_OBSTACLE = '🐝'
CHAR_PLAYER = '🐻'
CHAR_GOAL = '🍯'
# Path search related icons
CHAR_TRAVELLED = '⭐️'
CHAR_TO_VISIT = '❔'
CHAR_VISITED = '❌'

# Cost for walls or objects that are impossible to cross
UNAFFORDABLE_COST = 1000
ENEMY_COST = 150

# Max number of iterations to complete the quest
MAX_ITER = 100000

In [21]:
class Maze:

    # To store our maze 2D array
    maze_map = []
    heuristic_map = []
    start_pos = ()
    goal_pos = []
    size = ()

    # Rules for moving in the map
    ALLOWED_MOVES = (
        (1, 0),  # go right
        (0, 1),  # go down
        (0, -1),  # go up
        (-1, 0),  # go left
    )

    def __init__(self, file=None, maze_map=[]):
        self.maze_map = maze_map
        if file:
            self.parse_file(file)

        if len(self.maze_map) > 1 and len(self.maze_map[0]) > 1:
            self.pre_process()

    def parse_file(self, file):
        y = 0
        with open(file, 'r') as file:
            for line in file:
                x = 0
                _line = []
                line = line.strip()
                for c in line:
                    if c == 'S':
                        self.start_pos = (x, y)
                    elif c == 'G':
                        self.goal_pos.append((x, y))
                    _line.append(c)
                    x += 1
                y += 1
                self.maze_map.append(_line)
        self.size = (len(self.maze_map[0]), len(self.maze_map))
        print(self.goal_pos)
        print(f"Parsed '{file.name}' as a {self.size[0]}x{self.size[1]} map.")

    # Generate heuristics
    def pre_process(self): 

        self.heuristic_map = [[0 for i in range(self.size[0])] for j in range(self.size[1])]
        
        for goal in range(0, len(self.goal_pos)):

            for y in range(0, self.size[1]):

                for x in range(0, self.size[0]):

                    c = self.maze_map[y][x]

                    cost = self.heuristic_map[y][x]

                    # this is a wall
                    if c == '*':
                        cost = UNAFFORDABLE_COST
                    else:
                        # calculate euclidean distance based cost from goal to current point
                        _dist_cost = sqrt((self.goal_pos[goal][0] - x) ** 2 + (self.goal_pos[goal][1] - y) ** 2)
                        cost += _dist_cost
                        # A dangerous enemy!
                        if c == 'E':
                            cost += ENEMY_COST
                            self.heuristic_map[y - 1][x] += ENEMY_COST
                            self.heuristic_map[y - 1][x - 1] += ENEMY_COST
                            self.heuristic_map[y - 1][x + 1] += ENEMY_COST
                            self.heuristic_map[y + 1][x] += ENEMY_COST
                            self.heuristic_map[y + 1][x - 1] += ENEMY_COST
                            self.heuristic_map[y + 1][x + 1] += ENEMY_COST
                            self.heuristic_map[y][x - 1] += ENEMY_COST
                            self.heuristic_map[y][x + 1] += ENEMY_COST

                    # set total cost to the position
                    self.heuristic_map[y][x] = cost

    def get_cost_at_position(self, position: tuple):
        return self.heuristic_map[position[1]][position[0]]

    def print_heuristics(self):
        print('Heuristic cost map')
        for line in self.heuristic_map:
            for cost in line:
                if cost > 0:
                    _max_value = 250
                    _scale = int(floor((min((cost, _max_value)) / _max_value) * 255))
                    print(color_text(_scale, 0, 50, u"\u2589"), end='')
                else:
                    print(color_text(0, 0, 50, u"\u2578"), end='')
            print()


    def print_maze(self, path=None, step=0):
        """
        :param path: optional path to visualized
        :param step: viz up to step in the path starting 1. 0 will show all the path
        :return:
        """

        output_str = ''

        maze_map = self.maze_map.copy()

        # path available to print, if not print initial map
        if path:

            # if step is 0, visualize all the path
            if step == 0:
                step = len(path)

            # we update the maze with the path
            for i in range(0, step):
                pos = path[i]
                maze_map[pos[1]][pos[0]] = '.'

            # last position of the path is always the player
            pos = path[step-1]
            maze_map[pos[1]][pos[0]] = 'S'



        x, y = 0, 0
        for line in maze_map:
            for c in line:
                # this is a wall
                if c == '*':
                    _char_to_print = CHAR_WALL
                # The start position
                elif c == 'S':
                    _char_to_print = CHAR_PLAYER
                # the goal or end position
                elif c == 'G':
                    _char_to_print = CHAR_GOAL
                # An obstacle!
                elif c == 'E':
                    _char_to_print = CHAR_OBSTACLE
                # this is to print a position travelled in a path for resolved mazes
                elif c == '.':
                    _char_to_print = CHAR_TRAVELLED
                # empty space
                else:
                    _char_to_print = CHAR_EMPTY
                output_str += _char_to_print
                # print(_char_to_print, end='')
                x += 1

            # print()
            output_str += '\n'
            y += 1

        print(output_str)


In [22]:
class PositionNode:

    def __init__(self, parent: "PositionNode", position: tuple):
        self.cost = 0
        self.parent: "PositionNode" = parent
        self.position: tuple = position

    def __eq__(self, other: "PositionNode"):
        return self.position == other.position

In [23]:
class MazeSolver:

    def __init__(self, maze: Maze):

        self.maze = maze


    def create_path_from_node(self, node):

        # list of positions
        path = []

        while node:
            path.append(node.position)

            node = node.parent

        # reverse the list, so the last node added is the first move
        path = path[::-1]
        return path

    def a_star_search(self):
        # Setup start and initialize
        start_node = PositionNode(None, self.maze.start_pos)

        self.to_visit = []
        self.visited = []
        self.goals = []
        
        for node in range(0, len(self.maze.goal_pos)):
            self.goals.append(PositionNode(None, node))
        
        self.to_visit.append(start_node)

        # avoid infinite loop with maximum interations
        iteration = 1

        # main search loop while there are still nodes to visit
        for goal_node_position, goal_map_position in zip(self.goals, self.maze.goal_pos):
            print(Fore.RED, f'# Searching best path using A* for goal={goal_map_position}')
            while len(self.to_visit) > 0:

                current_node = self.to_visit[0]
                current_index = 0

                # iterate pending nodes to search lowest cost
                for i in range(len(self.to_visit)):
                    node = self.to_visit[i]
                    if node.cost < current_node.cost:
                        current_node = node
                        current_index = i

                iteration += 1
                if iteration > MAX_ITER:
                    print(' - Error: Too many iterations...')
                    # return partial result
                    return self.create_path_from_node(current_node)

                # Remove from pending, and add to visited
                self.to_visit.pop(current_index)
                self.visited.append(current_node)
                
                # Check if we've found the goal!
                #print(current_node.position, goal_node_position, sep='   ')
                if current_node == goal_node_position or current_node.position == goal_map_position:
                    print(Fore.RED, f' - Done! after {iteration} steps. Total cost was {current_node.cost}')
                    if (goal_node_position == self.goals[-1]):
                        return self.create_path_from_node(current_node)
                    else:
                        maze.print_maze(self.create_path_from_node(current_node), 0)
                        break

                # find children of current node in neighboring position using allowed moves
                for move in self.maze.ALLOWED_MOVES:

                    # Calculate new position
                    next_pos = (current_node.position[0] + move[0], current_node.position[1] + move[1])
                    # get cost
                    _cost = self.maze.get_cost_at_position(next_pos)

                    # is it possible to move there?
                    if _cost >= UNAFFORDABLE_COST:
                        continue

                    # Create new child node and add to the list
                    child = PositionNode(current_node, next_pos)
                    # calculate accumulated cost to move to that position
                    child.cost = _cost + current_node.cost

                    # if the child is in the list AND the total cost is not lower, don't add again
                    if len([i for i in self.to_visit if i == child and child.cost >= i.cost ]) > 0:
                        continue
                    self.to_visit.append(child)

In [24]:
if __name__ == '__main__':

    fn1 = 'maps/maze.txt'
    fn2 = 'maps/maze_small.txt'
    fn3 = 'maps/maze_difficult.txt'
    fn4 = 'maps/impossible_map.txt'
    fn5 = 'maps/maze_multiple.txt'
    fn6 = 'maps/maze_small_multiple.txt'

    # create the maze object
    maze = Maze(fn6)

    # print initial maze
    maze.print_maze()

    # maze solver
    solver = MazeSolver(maze)
    path = solver.a_star_search()

    # print solution for all steps
    maze.print_maze(path, 0)


[(17, 1), (17, 5)]
Parsed 'maps/maze_small_multiple.txt' as a 20x7 map.
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️🐻⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🍯⬛️⬛️
⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾⬛️⬛️⬛️🐾🐾⬛️⬛️
⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾⬛️⬛️⬛️⬛️🐾🐾⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🍯⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️

 # Searching best path using A* for goal=(17, 1)
  - Done! after 145 steps. Total cost was 270.62360102719896
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⭐️⬛️🐾🐾🐾🐾🐾🐾🐾🐾⭐️⭐️⭐️⭐️⭐️🐻⬛️⬛️
⬛️⬛️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⬛️⬛️⬛️🐾🐾⬛️⬛️
⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾⬛️⬛️⬛️⬛️🐾🐾⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🍯⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️

 # Searching best path using A* for goal=(17, 5)
  - Done! after 162 steps. Total cost was 275.0230752393446
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️
⬛️⬛️⭐️⬛️🐾🐾🐾🐾🐾🐾🐾🐾⭐️⭐️⭐️⭐️⭐️🐻⬛️⬛️
⬛️⬛️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⬛️⬛️⬛️🐾🐾⬛️⬛️
⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐾🐾⭐️⭐️⭐️⭐️⭐️⭐️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾⬛️⬛️⬛️⬛️🐾⭐️⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️🐾🐾🐾🐾🐾🐾🐾🐾🐻⬛️⬛️
⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️